# Dealing with Schema Changes in a Data Service

You are an Engineer at a small data company and you've been assigned to create a simple Bug Tracker for your Engineering Team.
In this lab, we'll go over the basic steps to get this Bug Tracker Desk, and how to deal with migrations as project scope chanages.
The final product will be a small tickting database that can be interacted with using python.
No knowledge of 
Ready to dive in?
Let's begin!

## Database Schema and Design

This bug tracker will have a database called _bug_tracker_, and within it, a table called _tickets_.
To start _tickets_ will have the following attributes:

- **ID**: A unique ID for each ticket
- **Title**: Overview of the issue.
- **Description**: Details about the issue.
- **Date Created**: When the issue was created and inserted into the database.

```sql
CREATE TABLE tickets (
  ticket_id INT AUTO_INCREMENT PRIMARY KEY
  title VARCHAR(32)
  description TEXT
  date_created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) ENGINE=InnoDB
```

This is enough for users to log issues into the database.
Using the schema above, let's create this table in the database using the python code below, but first you'll need to retrieve the Cloud SQL MySQL database IP address.

### Retreiving the Cloud SQL MySQL Database Host

From the [GCP Console](https://console.cloud.google.com/) search bar at the top of the page, type in SQL. Click the SQL result to open the Cloud SQL console view.

Observe the bugtracker-db Cloud SQL MySQL instance that has been created for you by the Cloud Academy lab environment.

It can take 10 minutes from when you started the lab for the instance to reach the ready status.

Copy the Public IP address and use it to overwrite the default host IP address in the code block in the following section.

### Creating the MySQL Database and Table

In [1]:
# create database and table schema
! pip install mysql-connector

import mysql.connector

ticket_table_schema = '''
CREATE TABLE IF NOT EXISTS tickets (
  ticket_id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(32),
  description TEXT,
  date_created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) ENGINE=InnoDB
'''

# make sure the host matches your GCP instance along with the password created earlier.
host = "34.94.61.132"
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()

# always start fresh. remove the database if it exists
cursor.execute("DROP DATABASE IF EXISTS bug_tracker")

# create the database within MySQL
cursor.execute("CREATE DATABASE bug_tracker")

# tell MySQL to use the database and any operations are applied to it
cursor.execute("USE bug_tracker")

# create the schema described above
cursor.execute(ticket_table_schema)

# confirm the table was successfully created
cursor.execute("SHOW TABLES")
print('tables:', [t[0] for t in cursor.fetchall()])

# clean up
cursor.close()
db.close()

     |████████████████████████████████| 11.9 MB 4.9 MB/s eta 0:00:01
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247949 sha256=e6b534395836f596518ceaed547285c784c41c47a97d1deaae2d7121dc4615cc
  Stored in directory: /home/jupyter/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql-connector
tables: ['tickets']


## Creating Tickets

With our database created, let's move on to creating a ticket and putting it in the database.
In python we'll create a `class` to store our ticket data, consiting of only a `title` and a `descriptoion`.
Remember, the `date_created` defaults to the current time and `ticket_id` is automatically generated.
After we've created the tickets and inserted them into the database, we can retreive the data, and populate them into our python class.
This makes working with the data afterwards, much easier as we can interact with the class properties instead of raw SQL data.

In [2]:
# connect to database
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker"
)

# grab cursor
cursor = db.cursor()


class Ticket:
    """
    Ticket class to house all ticket data
    Can be useful with retreiving data from the database as well
    Set ticket_id and date_create to optional args to they're not required for ticket creation,
    but usable in ticket retreival
    """
    def __init__(self, title, description, ticket_id = None, date_created = None):    
        self.ticket_id = ticket_id
        self.title = title    
        self.description = description
        self.date_created = date_created
    

def insert_ticket(ticket):    
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """    
    cursor.execute('INSERT INTO tickets(title, description) VALUES (%s, %s)', (ticket.title, ticket.description))    
    
# create some tickets    
t1 = Ticket('[FRONT-END] Login Btn Disabled', 'Go to the main login page, no matter what, the login button is never enabled')    
t2 = Ticket('[BACK-END] Missing id in JSON', 'When querying /user the JSON response is not returing the id along with the other properties')

# insert the first ticket!
insert_ticket(t1)    
insert_ticket(t2)

# view all tickets in the database
cursor.execute("SELECT * FROM tickets")
tickets = []
for t in cursor.fetchall():
  db_ticket = Ticket(t[1], t[2], t[0], t[3])
  tickets.append(db_ticket)

# print the ticket info
for t in tickets:
  print('({}) - {}\nDate Created: {}\n\t{}\n'.format(t.ticket_id, t.title, t.date_created.strftime("%m/%d/%Y, %H:%M:%S"), t.description))

# clean up        
cursor.close()
db.close()

(1) - [FRONT-END] Login Btn Disabled
Date Created: 07/06/2020, 00:00:47
	Go to the main login page, no matter what, the login button is never enabled

(2) - [BACK-END] Missing id in JSON
Date Created: 07/06/2020, 00:00:47
	When querying /user the JSON response is not returing the id along with the other properties



**Bonus**: Now that you have an idea of how we can create and insert tickets, try creating a third ticket `t3` and add it to the database.

## Scope Changes

Management is thrilled with your first iteration of the Bug Tracker!
However they have a few issues they would like you to address.
There will always more additions and updates so you need to ask yourself this question: _as the project evolves, how do we handle [migrations](https://en.wikipedia.org/wiki/Schema_migration)?_
Migrations are when changes need to be made to the database.
It usually referes to changes in a version-controlled manner, but we'll be ignoring that for now, as that is out of the scope of this exercise.

### Updating and Adding Columns

The first two tasks you are required to do is:

- Change the length of `title`. In the previous example the title `[FRONT-END] Login Btn Disabled` fits the title length. However `[FRONT-END] Login Button Disabled` would not. Change it to a length of `128` characters. This should resolve any issues of title length.
- Add a new column to the table: `project`. Instead of adding _[FRONT-END]_ or _[API]_ to the title, these can be their own column in the database. Note, that a project will now be required when inserting a ticket into the system.

To make these changes, we use the `ALTER` statement. The [ALTER statement](https://en.wikipedia.org/wiki/Data_definition_language#ALTER_statement) makes changes to a database object. Using this will alter the `title` column in the table, and we'll also alter the `tickets` table itself to add in our new column, _projects_.
To add a new column, the syntax in MySQL is `ALTER TABLE <table-name> ADD COLUMN <column-name> <column-type>`.
To change a column we use `MODIFY`: `ALTER TABLE <table-name> MODIFY COLUMN <column-name> <new-column-type>`.
When finished, we'll use MySQL's [`DESCRIBE` statement](https://dev.mysql.com/doc/refman/8.0/en/describe.html) to check that the data has been updated properly.

In [3]:
# connect to database
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker"
)

# grab cursor
cursor = db.cursor()

# make changes to the database
# modify title
cursor.execute('ALTER TABLE tickets MODIFY COLUMN title VARCHAR(128)')    
# add project
cursor.execute('ALTER TABLE tickets ADD COLUMN project VARCHAR(64) NOT NULL AFTER title')    

# check that the data was updated properly
# look for project and title
cursor.execute('DESCRIBE tickets')    
for row in cursor.fetchall():    
    print(row[0], '-', row[1])   
    
cursor.close()
db.close()

ticket_id - int(11)
title - varchar(128)
project - varchar(64)
description - text
date_created - timestamp


With these changes in place, our previous code no longers works.
Take a moment and try to update the `Ticket` class along with the `insert_ticket` function.
You can find the updated code below:

```python
class Ticket:    
    """    
    Ticket class to house all ticket data.
    Can be useful with retreiving data from the database as well.
    Set ticket_id and date_create to optional args to they're not
    required for ticket creation, but usable in ticket retreival. 
    """    
    def __init__(self, title, project, description, ticket_id = None, date_created = None):
        self.ticket_id = ticket_id    
        self.title = title
        self.project = project
        self.description = description
        self.date_created = date_created
    
    
def insert_ticket(ticket):    
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """    
    cursor.execute('INSERT INTO tickets(title, project, description) VALUES (%s, %s)', (ticket.title, ticket.project, ticket.description))    
```

### Adding a Relationship

This ticket system is looking better every iteration!
Unfortunately, management has come to you again with another request.
They want to be able to assign tickets to staff in the Engineering department.
They also want to be able to manage the users in the database.

To achive this new functionality, we'll need to create a _users_ table and have a [one-to-many relationship](https://en.wikipedia.org/wiki/One-to-many_(data_model)) between a user and tickets.
This makes sense as a single user could have multipe tickets assigned to them.
Our schema for the users will look like the following:

```sql
CREATE TABLE IF NOT EXISTS users (
  user_id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(30),
  email VARCHAR(30) UNIQUE,
) ENGINE=InnoDB
```

We'll also need to update the `tickets` table again and the `user_id` as `assignee` using the following python/SQL code:

```python
fk_sql_code = '''
ALTER TABLE tickets ADD COLUMN assignee INT,
    ADD FOREIGN KEY fk_assignee_user_id(assignee) REFERENCES users(user_id)
'''

cursor.execute(fk_sql_code)
```

Look at the following python example to see how it all ties together.

In [ ]:
users_table_sql = '''
CREATE TABLE IF NOT EXISTS users (
  user_id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(30),
  email VARCHAR(30) UNIQUE
) ENGINE=InnoDB
'''

fk_sql_code = '''
ALTER TABLE tickets ADD COLUMN assignee INT AFTER project,
    ADD FOREIGN KEY fk_assignee_user_id(assignee) REFERENCES users(user_id)
'''

# connect to database
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker"
)

# grab cursor
cursor = db.cursor()

# create the users table first
cursor.execute(users_table_sql)

# create the column and user --> tickets relationship
cursor.execute(fk_sql_code)

# check our work was successful
cursor.execute("SHOW TABLES")    
print('tables:', [t[0] for t in cursor.fetchall()])    

cursor.execute('DESCRIBE tickets')    
for row in cursor.fetchall():    
    print(row[0], '-', row[1])
    
cursor.close()
db.close()

This is starting to look good! There are few more loose ends to tie up.

First, we need to create a `User` class in our python script to handle users.
This will be similar to the `Ticket` class we created earlier, matching the class properties with the ones from the database schema.

Second, add in a few users.
We will build off our `insert_ticket` function from before and create a new one called `insert_user`.

Third, we need to update the the `Ticket` class to include our assignee (`user_id`).
This will be a reference to `user_id`, not the `User` class, as that is out of scope for this exercise.

Finally, back log our current tickets and populate users for them.
For this, we will use the `UPDATE` statement in MySQL and update all tickets with the id for the first user we created.

Try to finish the python code below to accomplish these tasks.
The full source code will be displayed afterwards.

In [ ]:
# make sure to complete the insert_user function
# along with the update_sql SQL statement
# otherwise the script below will not work


# connect to database
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker"
)

# grab cursor
cursor = db.cursor()

# finish populating the User class
class User:    
    """    
    User class to store users to/from the database    
    """    
    def __init__(self, name, email, user_id=None):
        self.name = name    
        self.email = email    
        self.user_id = user_id    

# using insert_ticket from before, remove pass, and finish the function insert_user
def insert_user(user):
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """
    pass

# create users
user1 = User('Link', 'courage@hyrule.world')
user2 = User('Zelda', 'wisdom@hyrule.world')

# insert users into database
insert_user(user1)
insert_user(user2)

# view users
cursor.execute('SELECT * FROM users')
print('users:', cursor.fetchall())

# assign all previous tickets
# first grab the user_id of user1
cursor.execute('SELECT user_id FROM users WHERE email = "{}"'.format(user1.email))
# assign the id to user1
user1.user_id = cursor.fetchone()[0] # will fail without insert_user finished

# complete the update_sql query
# make sure to update the ticket table and set the assignee to the user_id
update_sql = 'use python formatting to create an UPDATE statement'.format(user1.user_id)
cursor.execute(update_sql)

# view all tickets
cursor.execute("SELECT * FROM tickets")
tickets = []
for t in cursor.fetchall():
  db_ticket = Ticket(t[1], t[2], t[0], t[3])
  tickets.append(db_ticket)
for t in tickets:
  print('({}) - {}\nDate Created: {}\n\t{}\n'.format(t.ticket_id, t.title, t.date_created.strftime("%m/%d/%Y, %H:%M:%S"), t.description))

#clean up
cursor.close()
db.close()

Solution for the code above

```python
# connect to database
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123",
  database="bug_tracker"
)

# grab cursor
cursor = db.cursor()

# finish populating the User class
class User:    
    """    
    User class to store users to/from the database    
    """    
    def __init__(self, name, email, user_id=None):
        self.name = name    
        self.email = email    
        self.user_id = user_id    

# using insert_ticket from before, remove pass, and finish the function insert_user
def insert_user(user):
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """
    cursor.execute('INSERT INTO users(name, email) VALUES (%s, %s)', (user.name, user.email))

# create users
user1 = User('Link', 'courage@hyrule.world')
user2 = User('Zelda', 'wisdom@hyrule.world')

# insert users into database
insert_user(user1)
insert_user(user2)

# view users
cursor.execute('SELECT * FROM users')
print('users:', cursor.fetchall())

# assign all previous tickets
# first grab the user_id of user1
cursor.execute('SELECT user_id FROM users WHERE email = "{}"'.format(user1.email))
# assign the id to user1
user1.user_id = cursor.fetchone()[0] # will fail without insert_user finished

# complete the update_sql query
# make sure to update the ticket table and set the assignee to the user_id
update_sql = 'UPDATE tickets SET assignee={}'.format(user1.user_id)
cursor.execute(update_sql)

# view all tickets
cursor.execute("SELECT * FROM tickets")
tickets = []
for t in cursor.fetchall():
  db_ticket = Ticket(t[1], t[2], t[0], t[3])
  tickets.append(db_ticket)
for t in tickets:
  print('({}) - {}\nDate Created: {}\n\t{}\n'.format(t.ticket_id, t.title, t.date_created.strftime("%m/%d/%Y, %H:%M:%S"), t.description))

#clean up
cursor.close()
db.close()
```

## Complete Code Set

Below is a start to finish of the codebase and changes made throughout this lab.
Be sure to tinker around with this and figure out how it all works.

**Bonus Challenges**
Update the code below to do the following:

- Update the Ticket class to have a `user_id` property. Update the `insert_ticket` function to insert the `user_id` from the Ticket object. This will take a bit of reworking of the code, but will set you up nicely for the following challenges.
- Create a new function `insert_ticket_with_user` that takes in an email address and a ticket object. Query the database for the `user_id` of that email and insert the ticket properly.
- Create a new user with the same email address as one already in the database. What happens? Why do you think this is the case?

```python
import mysql.connector

ticket_table_schema = '''
CREATE TABLE IF NOT EXISTS tickets (
  ticket_id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(32),
  description TEXT,
  date_created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
) ENGINE=InnoDB
'''

# make sure the host matches your GCP instance along with the password created earlier.
db = mysql.connector.connect(
  host=host,
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()

# always start fresh. remove the database if it exists
cursor.execute("DROP DATABASE IF EXISTS bug_tracker")

# create the database within MySQL
cursor.execute("CREATE DATABASE bug_tracker")

# tell MySQL to use the database and any operations are applied to it
cursor.execute("USE bug_tracker")

# create the schema described above
cursor.execute(ticket_table_schema)

# confirm the table was successfully created
cursor.execute("SHOW TABLES")
print('tables:', [t[0] for t in cursor.fetchall()])

class Ticket:
    """
    Ticket class to house all ticket data
    Can be useful with retreiving data from the database as well
    Set ticket_id and date_create to optional args to they're not required for ticket creation,
    but usable in ticket retreival
    """
    def __init__(self, title, description, ticket_id = None, date_created = None):    
        self.ticket_id = ticket_id
        self.title = title    
        self.description = description
        self.date_created = date_created
    

def insert_ticket(ticket):    
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """    
    cursor.execute('INSERT INTO tickets(title, description) VALUES (%s, %s)', (ticket.title, ticket.description))    
    
# create some tickets    
t1 = Ticket('[FRONT-END] Login Btn Disabled', 'Go to the main login page, no matter what, the login button is never enabled')    
t2 = Ticket('[BACK-END] Missing id in JSON', 'When querying /user the JSON response is not returing the id along with the other properties')

# insert the first ticket!
insert_ticket(t1)    
insert_ticket(t2)

# view all tickets in the database
cursor.execute("SELECT * FROM tickets")
tickets = []
for t in cursor.fetchall():
  db_ticket = Ticket(t[1], t[2], t[0], t[3])
  tickets.append(db_ticket)

# print the ticket info
for t in tickets:
  print('({}) - {}\nDate Created: {}\n\t{}\n'.format(t.ticket_id, t.title, t.date_created.strftime("%m/%d/%Y, %H:%M:%S"), t.description))

# make changes to the database
# modify title
cursor.execute('ALTER TABLE tickets MODIFY COLUMN title VARCHAR(128)')    
# add project
cursor.execute('ALTER TABLE tickets ADD COLUMN project VARCHAR(64) NOT NULL AFTER title')    

# check that the data was updated properly
# look for project and title
cursor.execute('DESCRIBE tickets')    
for row in cursor.fetchall():    
    print(row[0], '-', row[1])   

users_table_sql = '''
CREATE TABLE IF NOT EXISTS users (
  user_id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(30),
  email VARCHAR(30) UNIQUE
) ENGINE=InnoDB
'''

fk_sql_code = '''
ALTER TABLE tickets ADD COLUMN assignee INT AFTER project,
    ADD FOREIGN KEY fk_assignee_user_id(assignee) REFERENCES users(user_id)
'''

# create the users table first
cursor.execute(users_table_sql)

# create the column and user --> tickets relationship
cursor.execute(fk_sql_code)

# check our work was successful
cursor.execute("SHOW TABLES")    
print('tables:', [t[0] for t in cursor.fetchall()])    

cursor.execute('DESCRIBE tickets')    
for row in cursor.fetchall():    
    print(row[0], '-', row[1])

# finish populating the User class
class User:    
    """    
    User class to store users to/from the database    
    """    
    def __init__(self, name, email, user_id=None):
        self.name = name    
        self.email = email    
        self.user_id = user_id    

# using insert_ticket from before, remove pass, and finish the function insert_user
def insert_user(user):
    """    
    Insert the ticket into the database using its properties    
    Is vulnerable to SQL injection! For demonstration purposes only!    
    """
    cursor.execute('INSERT INTO users(name, email) VALUES (%s, %s)', (user.name, user.email))

# create users
user1 = User('Link', 'courage@hyrule.world')
user2 = User('Zelda', 'wisdom@hyrule.world')

# insert users into database
insert_user(user1)
insert_user(user2)

# view users
cursor.execute('SELECT * FROM users')
print('users:', cursor.fetchall())

# assign all previous tickets
# first grab the user_id of user1
cursor.execute('SELECT user_id FROM users WHERE email = "{}"'.format(user1.email))
# assign the id to user1
user1.user_id = cursor.fetchone()[0] # will fail without insert_user finished

# complete the update_sql query
# make sure to update the ticket table and set the assignee to the user_id
update_sql = 'UPDATE tickets SET assignee={}'.format(user1.user_id)
cursor.execute(update_sql)

# view all tickets
cursor.execute("SELECT t.ticket_id, t.title, u.email, t.date_created, t.description FROM tickets t INNER JOIN users u ON (t.assignee = u.user_id)")
for row in cursor.fetchall():
    print('({}) - {}\nAssignee: {}\nDate Created: {}\n\t{}\n'.format(row[0], row[1], row[2], row[3].strftime("%m/%d/%Y, %H:%M:%S"), row[4]))

#clean up
cursor.close()
db.close()
```